# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞
browser = webdriver.Chrome(executable_path='../other/chromedriver')

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

提示：可參考以下的 Stack Overflow: <br>
https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166

In [3]:
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁
timeout = 5
html = None
try:
    element_present = EC.presence_of_element_located((By.CLASS_NAME, 'infinite_scroll'))
    WebDriverWait(browser, timeout).until(element_present)
    
    #
    # 以下是用Selenium模擬下拉網頁動作，讓網頁更新
    #
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait to load the page.
        time.sleep(2)
        
        new_height = browser.execute_script("return document.body.scrollHeight")

        if (new_height > last_height): # not the end
            last_height = new_height
            print('網頁更新中...')
        else:
            html = browser.page_source
            browser.close()
            print('到達頁面底端')
            break
except TimeoutException:
    print("Timed out waiting for page to load")

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明： (重要 ===> 須注意)
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 「特別收益」Q＆A觀念全公開！善用配置訣竅分散風險　享有穩定高收益水準

▲面對全球高收益債在首季跌跤，安聯投信建議，投資人可考慮重整步伐，讓全球高收益債與特別收益「各配置一半」。（圖／pixabay）
記者楊絡懸／台北報導
面對低經濟成長率、低能見度、低利率及高波動已漸漸成為「市場新常態」，投資人展望2020下半年可鎖定第一流企業，納入以股利、債息兼具的「特別收益」（Preferred Securities）為收益型資產配置的標的。安聯投信針對「特別收益」解釋5項QA問答，提供投資人了解與參考。
Q：為什麼投資組合裡要納入「特別收益」？與主流的債券有何異同？
如果參考美林固定利率特別證券指數，這類特別收益債券近期平均到期殖利率約5%至6%，明顯高於全球公債的1.05%和投資等級債的2.82%，過去5年的波動度則介於全球投資等級債和全球公債之間；特別收益的發行者多為投資等級，與高收益企業或新興國家截然不同，不僅能分散違約風險，也能享有一定的收益水準。
▲安聯特別收益多重資產基金經理人林素萍（左）與安聯投信產品經理胡韡耀，深入剖析特別收益的投資利基，建議投資人鎖定第一流企業，布局特別收益也能有亮麗的息收。（圖／安聯投信提供）
Q：「特別收益」和「特別股」有哪些異、同之處？
Q：「特別收益」的前景為何？
美國特別收益證券市場之所以蓬勃發展，主要是這類企業所發行的各類特別收益證券，長期配發利息紀錄穩健，值此收益資產難尋的時刻，聚焦「特別收益」的多重資資產策略更易受到市場的注意。
▲「特別收益」（Preferred Securities）是兼具股、債特質的證券，同一家企業發行的特別收益，因為資本結構不同，票息往往比一般債券和股票高，也為企業與投資人創造雙贏。（圖／pixabay）
Q：發行「特別收益」證券的企業大多是哪類產業？
Q：安聯投信投資團隊挑選「特別收益」的考量為何？
【延伸閱讀】
----------------------------------------------------------------------

[1] 證交所：大同若能確保股東權益　有機會恢復股票正常交易　

記者陳心怡／台北報導
大同公

▲勞動部台灣就業通自即日起辦理為期1個月的「Invest Taiwan 投資臺灣三大方案企業線上就業博覽會」。（圖／資料照）
記者余弦妙／台北報導
畢業季過後，緊接而來就是求職潮，勞動部為協助青年順利求職，持續推動「應屆畢業青年就業措施」，鼓勵應屆畢業生投入就業市場，由台灣就業通自即日起辦理為期1個月的「Invest Taiwan 投資臺灣三大方案企業線上就業博覽會」，包含大立光電、矽品精密等200家企業，祭出超過1萬筆職缺，薪資最高上看7萬7。
勞動部透露，這次有參與徵才活動企業包含「大立光電」、「矽品精密」、「欣興電子」、「上銀科技」、「友達光電」、「台灣精銳」、「聖杰國際」等約200家廠商共釋出超過1萬筆的職缺，種類有程式設計工程師、自動控制工程師、研發人員、品管人員、倉管人員、客服人員以及儲備幹部等，薪資最高上看7萬7。
勞動部指出，這次由台灣就業通結合投資臺灣三大方案等事業單位，主要是針對回臺投資廠商、未曾赴中國投資的大企業，以及在地的中小企業，提供專案貸款、土地租金優惠等協助措施，加速廠商在臺安心投資。
求職者只要加入台灣就業通會員並於線上填寫求職履歷，即可在活動期間於活動頁面瀏覽、搜尋相關職缺並直接應徵媒合。活動後參展企業將結合台灣就業通網站持續辦理徵才，求職者均可依照個人就業規劃，於台灣就業通網站上設定職缺條件持續尋找理想的工作，亦可透過各地就業中心協助就業。
►過期票券別浪費！點這邊...幫你搶救變身
----------------------------------------------------------------------

[11] 影／三倍券申購突破百萬大關！搶搭數位加碼優惠「最多人選擇綁定這一款」

記者林淑慧／台北報導
振興三倍券今（1）日開放第一波預購，依據經濟部統計，截至下午五時為止，上官網申購的民眾已正式突破百萬大關，達121萬569人次；其中，信用卡業者祭出高額回饋金發威，已有超過34萬人次瘋搶信用卡早鳥優惠，成為最多民眾選用的綁定數位工具。
▲「振興三倍券」今（1）日開放預購及綁定數位工具，7月15日發放，官網網址為3000.gov.tw。（圖／翻攝自振興三倍券官網）
振興三倍券官網的數位綁定跟紙本預訂功能，今天正式解封上線，針對首日申購狀況，經濟部表示，目前全台預訂實體紙本券者約72萬7318人次，選擇於四大超

----------------------------------------------------------------------

[25] 藍弋丰／如何讓超昂貴的先進醫療　變得人人看得起？

● 藍弋丰／台大醫學系畢業，從事翻譯、圖文創作、業餘歷史研究，現任台灣民眾黨顧問。
衛福部疾病管制署副署長，也是防疫指揮中心發言人莊人祥，近來針對為何自費採檢要價7000元，霸氣回應：7000元不是只有檢驗試劑的成本費用，每次做聚合酶連鎖反應（PCR，目前篩檢病毒的主要手段）檢驗，負責的醫檢人員都需穿防護衣來做，也需要考量到負責檢驗的醫院成本，如果覺得不值得這個價錢，那他也無話可說。
說的真是太好了！不過才沒多久前，衛福部跟健保署認為醫材定價有所不同就是「亂象」，而不是霸氣的說：手術使用醫材需要整個醫療團隊的專業，也需要考量到營運手術室的醫院成本，「如果覺得不值得這個價錢，那也無話可說。」
健保企圖消滅「生產資料私有」　要醫院當慈善機構？
許多人不明白，為何此次醫材議題醫界全面反彈如此嚴重，為何還有大量非醫界專業人士全面聲援。這些不明白的人或許身體健康，沒有常常看病，因此不知道台灣的健保長期以來不斷的採取各種錯誤政策，使得醫療品質每況愈下，這次只是最後一根稻草。
十幾年來健保不斷壓低給付、「吃霸王餐」無理核刪，已經造成無數原廠藥品、先進醫材直接拋棄台灣市場，想買都買不到，2016年推出DRG惡政，更是雪上加霜，區域醫院只能另尋生存管道，健保又視為「亂象」。
▲ 自費醫材有差額上限，引起醫界不滿。（圖／記者徐斌慎攝）
如此異想天開的反市場共產主義烏托邦思維（共產主義的定義：一種共享經濟結合集體主義的思想，主張消滅生產資料私有制，進行集體生產，以階級鬥爭建立無階級的社會），施行十幾年下來，窮人有更看到病嗎？當然不可能，共產主義只會導致經濟崩潰，醫療也一樣。
其結果是醫療資源大量流失，好的藥消失，好的醫材消失，好的術式跟著消失，醫護人員無法得到最好的工具來照護病人，無法用最先進的對病人最好的辦法治療病人，反而是被迫用落伍的辦法來治療病人，這是醫護基層對健保惡政極度憤恨的原因。
較常去醫院的病人，以及產業訊息發達的專業人士，也都感受得到醫療資源正在快速流失，所以在此次群起聲援醫護。
逼迫醫院虧損 → 醫療資源消失、品質下降
市場給予獲利，就會鼓勵資源投入，市場讓人虧損，就

▲台股終場收在11703點。（資料照／記者湯興漢攝）
記者陳心怡／台北報導
美股四大指數全數收高，激勵台股今 (1) 日開高走高，且呈現股匯齊揚格局，盤中最高來到11736點，終場收在11703點，上漲82點或0.71%，成交量1920.74億元。
蘋概三王中，台積電（2330）以317.5元作收、上漲1.44%；鴻海（2317）以85.9元作收、下跌0.46%；股王大立光（3008）以4200元作收、上漲2.82%。
大型權值股中，台塑化、台塑收在平盤，中華電及聯發科分別下跌0.43%和0.35%。
國泰金、富邦金今天除息大秀打頭陣，兩檔龍頭股都穩步邁向填息路，國泰金上漲超過 1% 尤為亮麗，其餘金融股也都同步走揚，兆豐金、台新金、上海商銀都漲逾 1%。
今日上市各類股以光電類股表現最佳，上漲2.12%，表現較差者為玻璃股類股，下跌1.19%。
此外，大同公司董事改選爆發爭議，證交所認為大同未能具體說明相關問題，決定自 7 月 2 日起將大同打入全額交割股，衝擊大同今天股價重挫跌停，來到 18.9 元，創 3 個月新低價。
----------------------------------------------------------------------

[36] 錢世傑／把老闆炒魷魚的第二人生

▲勞保基金面臨破產窘境。（示意圖／記者李毓康攝）
●錢世傑Dr. J／低調的法律及投資研究者
下一個開刀的對象是勞保基金
早上在便利商店7-11裡頭打字，旁邊突然來了兩位老人，談吐之間有著文人的氣質，只見一個老人正在埋首簽署一頁文件，另一位老人正在輕聲細語地述說著目前的退休金困境，似乎埋首簽署文件的那位老人正在簽署授權書，拜託能有個管道爭取一些退休金上的權益。
由於人口結構惡化，退休老人活太久引發的財政困境，政府只好對軍公教下了重手、砍了退休金，許多人奮鬥一生所期待的退休金都被打折，心中自然也有所不滿，可是大法官會議認為可以調整並未違反憲法，少了退休金的老人也只能徒呼負負、無能為力；非軍公教的其他民間階層也是紛紛指著這些軍公教的鼻子罵，認為這些老人退休金福利這麼好，少個幾萬也要到處吵鬧，真是貪婪啊！
▲反年改抗議群眾。（圖／記者季相儒攝）
奮鬥多年的目標，卻是一個「歸零斷崖」
說完了退休金被砍的殘酷，讓我接著說「歸零斷崖」。
各位試想看看，當你拼到60歲，終於

記者陳心怡／台北報導
大同 (2371) 股東會引發爭議，證交所認為大同未能具體說明相關問題，決定自7月2日起，將大同有價證券變更交易方法為全額交割，利空衝擊大同今 (1) 日股價一開盤隨即被打到跌停，委賣張數逾1.3萬張。
大同今天在市場賣壓出籠下，開盤即重挫跌停，摜破新台幣20元關卡，股價來到18.9元，下跌2.1元，跌幅10%，並創3個月新低價。
大同昨(30)日召開股東常會，改選董事，由董事長林郭文艷領軍的公司派，與由三圓建設董事長王光祥代表的市場派競爭激烈，結果由公司派9:0全勝，取得6席一般董事及3席獨立董事。
大同昨晚也召開重大訊息說明，董事長林郭文艷表示，股東持股比例超過10%未依法申報，不得行使表決權、選舉權，據統計共有 53.32% 股權。
證交所表示，大同召開重大訊息說明記者會，仍未能具體說明今日召開的股東會中，逕行認定部分股東所持股份無表決權、未發給部分股東表決權票及選舉票等相關法律依據及事證，將大同交易方法變更為全額交割股。
受市場派提名獨董的前時代力量立委黃國昌在臉書發文表示，大同股價直接跳空跌停，是林郭集團違法亂搞，成就的是一己之私，受害付出代價的，卻是無辜的股東與投資大眾，如此嚴重傷害臺灣資本市場秩序的惡質手法，令人不齒，也再次呼籲公司治理與市場秩序，攸關大家共同的福祉，不要再讓政治的顏色蒙蔽了基本的理智，縱容這樣的惡行。
----------------------------------------------------------------------

[44] 6月PMI回升2.4個百分點至47.2％　中經院：長期還是要看國際經濟情勢

▲中經院院長張傳章。（圖／記者余弦妙攝）
記者余弦妙／台北報導
中華經濟研究院今（1）日公布6月臺灣製造業採購經理人指數（PMI），經季節調整後指數已連續3個月呈現緊縮，指數由2012年7月創編以來最快緊縮速度（44.8%）回升2.4個百分點至47.2%。學者認為，廠商多認為景氣有慢慢回溫，但還未回復到疫情發生前，長期仍要看國際經濟情勢。
中經院指出，在五項組成指標中，新增訂單、生產與人力僱用持續緊縮，供應商交貨時間上升，存貨轉為緊縮。 2020年6月六大產業中，四大產業回報PMI為緊縮，依緊縮速度排序為電力暨機械設備產業（40.2%）、交通工具產業（41.9%）、基礎原物料產業（45.

圖、文／富蘭克林提供
新冠疫情危機尚未完全解除，美中又掀起另一波政治與經濟角力，明明全球市場利空不斷，但全球股市卻在劇烈波動下緩步反攻，美股三大指數有機會收復今年以來失土。面對如此矛盾衝突又詭譎多變的投資環境，與其大膽賭時機，不如安心「存基金」。
富蘭克林證券投顧根據2020年國民理財大調查中的投資人痛點，設計並推出了「存基金」線上投資方案，結合國民理財機器人、定期定額與專業理財顧問3大優勢，從搭配投資組合到資產再平衡，乃至於有紀律的長期投資，完全e指搞定。無須賭時機，也有機會創造投資好成績。
沒人敢打包票疫情會不會再次爆發，也沒人知道明天股市是漲是跌，唯一能確定的是市場長期趨勢向上！只要秉持「整組買、持續買、不亂賣」的存基金九字箴言，就有機會透過時間提高投資勝率與報酬率，堪稱最人性化的投資工具。目前系統已於7月1日正式上線、開放下單，想要透過存基金累積第一桶金的人千萬別錯過！
----------------------------------------------------------------------

[54] 普生新冠肺炎試劑　獲中研院抗原及抗體檢測試劑授權合約

▲普生總經理林孟德。（圖／普生提供）
記者姚惠茹／台北報導
普生（4117）今（1）日宣布已與中央研究院完成簽訂新冠肺炎抗原快篩檢測的全球非專屬授權合約，以及新冠肺炎抗體快篩檢測試劑的台灣區授權合約，並積極規劃抗原/抗體快篩試劑向台灣食藥署(TFDA)提出專案製造申請許可，加速推進普生旗下新冠肺炎試劑的國內外檢測試劑市場布局。
普生表示，公司擁有新冠肺炎試劑健全產品陣容，包括RT-PCR核酸試劑、抗原/抗體快篩試劑與ELISA血清抗體試劑等產品，其中與中研院合作研發抗原快篩病毒檢測試劑，可比照流感快篩，檢測時間僅需15至20分鐘內，即可初步篩檢出陽性與否，適用於機場、診所、社區群聚感染等大規模檢測需求，有助於分攤試劑檢測量能，及時進行隔離分流作業，強化各國疫情監管能力提升。
普生指出，普生旗下GB SARS-CoV-2 Real-Time RT-PCR核酸病毒檢測試劑於國外申請許可認證進展上，不僅已陸續取得歐盟CE認證、印度進口許可證，近日更與澳洲客戶Labtest Medical Pty Ltd合作取得澳洲衛生部醫療用品管理局(TGA)核准澳洲治療產品(ARTG)IVD CLASS 

▲台股早盤上漲。（示意圖／記者湯興漢攝）
記者王曉敏／台北報導
經濟數據改善提升華爾街信心，美股周二收高。台股今（1）日以11622點開出，隨後指數來到11647點，上漲26點。
大同（2371）昨日舉行股東常會改選董事，由公司派拿下9席董事。證交所昨晚公告，大同昨日召開的重大訊息說明會能具體說明在會中逕行認定部分股東持股無表決權、未發給部分股東表決權票等，宣布大同股票於7月2日起實施全額交割。大同今日開盤跳空跌停，委賣高達4萬多張，股價直接摜破20元關卡。
蘋概三王中，台積電（2330）以315元開出，上漲0.64%；鴻海（2317）開在86.5元，上漲0.23%；股王大立光（3008）以4135元開出，上漲1.22%。
美股三大指數6月底成功繳出漂亮成績單，分別創下33年、22年和21年的最大單季漲幅。儘管疫情持續，但經濟激勵政策、經濟重啟、疫苗研發和房市表現佳等因素，都讓美股在6月衝上高點。
周二美股表現，道瓊工業指數終場上漲217.08點，或0.85％，收在25812.88點；那斯達克指數上漲184.61點，或1.87％，收在10058.77點；標普500指數則是中場上漲47.05點，或1.54％，以3100.29點作收。
----------------------------------------------------------------------

[65] 就是不還！欠稅大戶全是老面孔　美麗華集團黃春福重新上榜

▲財政部公布的欠稅大戶名單中，只有一人是新面孔。（圖／記者陳韋帆攝）
記者吳靜君／台北報導
財政部今（1）日公告欠稅大戶，全是老面孔，包含全國前20大、企業前20大以及個人前20大，根據財政部的統計，只有一起是新面孔，為個人案件中是美麗華集團德安美麗信事業由黃榮華之子黃春福，欠稅金額為3.29億元，因為原本分期繳，後因故未繳重回榜上。
根據財政部賦稅署彙整統計，公告件數956件，金額約982.25億餘元，與108年公告資料相較，件數及金額分別增加18件、16.72億餘元，
根據財政部公告，今年全國前20名，環亞飯店為首，欠稅金額達24.61億元、黃任中之子黃若谷居次，19.04億元，黃任中第三，欠稅18.94億元；高雄的黃承志、黃瑞月、黃瑞雲14.87億元，居第四；黃任中姊姊夏黃新平則是12.37億元，居於第五。
在企業方面，名單

▲大同公司雖然召開重大訊息說明會，董事長林郭文艷也親自出面，但證交所還是認為沒有具體說明，決定改列大同股票為全額交易股。（圖／記者陳心怡攝）
財經中心／台北報導
大同股東們注意了，證交所昨（30）晚宣布，大同股票自7月2日起改列全額交割股。換句話說，今天將是大同股票正常交易的最後一天，市場如何反映解讀此事，等台股今天開盤就會有答案。
證交所表示，大同公司（2371）於6月30日召開之重大訊息說明記者會，未能具體說明其於109年召開之股東常會運行認定部分股東所持股份無表決權、未發給部分股東表決權票及選舉票之法律依據及相關事證，對股東權益影響甚鉅，並與公司治理股東行動主義有違，核有本公司營業細則第49條第1項第14款規定之情事，自109年7月2日起，將其上市有價證券列為變更交易方法，也就是打入全額交割股。
▲大同經營權之爭延燒，市場派在股東會後召開臨時記者會，由市場派代表王光祥（圖中）、蔡玉真、白旭屏、莊正及周亞恬出席。（圖／記者湯興漢攝）
----------------------------------------------------------------------

[76] 燦星旅新主首度露面！侯佑霖：開發歐洲遊艇、帆船等海洋觀光行程

▲亞果遊艇集團暨燦星旅董事長侯佑霖。（圖／燦星旅提供）
記者林淑慧／台北報導
燦星旅行社日前易主，由台南幫第三代、亞果遊艇集團董事長侯佑霖出線，他今（30）日首度公開露面，表示個人收購燦星旅後不會更名，將借助燦星旅在東北亞線與OTA之優勢，並開發歐洲遊艇、帆船等與海洋相關的行程，打造「海上遊憩專家」的新亮點，以期與亞果遊艇高端會員經營發揮互補效果。
原為燦坤集團旗下的「燦星旅行社」於今年三月公告將五成股權出售給台南幫第三代侯佑霖及鄭寶蓮。侯佑霖從創設萬通人力公司，目前旗下已有亞果國際潛水、亞果遊艇開發、亞澎遊艇開發、亞青海洋文創、錚儀遊艇、樂活海洋學院等六大集團企業，短短五年間躍居我國遊艇產業的明日之星，這次入主燦星旅也引發業界高度關注。
亞果遊艇30日舉行興櫃前法人說明會，侯佑霖率領經營團隊簡報營運展望，他於會後受訪時表示，今年以來新冠肺炎疫情衝擊全球，觀光產業成為海嘯第一排的重災區，剛好燦坤集團想回歸本業並處分非本業業務進而洽談，收購的價格不錯且正是能好好整頓的時機，目前規劃先私募再減資四成，透過挹注資金來重整公司

▲大樂透端午加碼，53人一注獨得百萬，尚餘40組。（圖／記者鄒鎮宇攝）
記者柯沛辰／台北報導
大樂透連槓3期，頭獎上看1.2億元。台彩推出端午加碼「100組100萬」部分，至今已有53名幸運兒獨得百萬，目前尚有40組未開出，今晚將持續加開。
端午加碼100組百萬部分，40組分別為：
第1組-第4組
第5組-第8組
第9組-第12組
第13組-第16組
第17組-第20組
第21組-第24組
第25組-第28組
第29組-第32組
第33組-第36組
第37組-第40組
＊實際獎號以台彩公布為主
台彩大樂透端午加碼活動，採「當期買，對當期」方式進行，無法跨期兌獎，只要對中6個加開獎號，每組號碼獎金100萬元，若同組有多注中獎，依注數均分，直到全數送出或7月10日活動結束為止。
----------------------------------------------------------------------

[86] 港版國安法通過！　國銀在香港曝險大減581億

▲港版國安法今天正式通過，外界關注香港國際金融中心地位是否連帶受到衝擊。（示意圖／路透）
記者紀佳妘／台北報導
港版國安法今（30）日正式通過，外界關注香港國際金融中心地位是否受到影響。金管會最新公布國銀在香港曝險情況，到今年5月底，國內銀行在香港曝險金額下降至6,748億元，較去年同期大減581億元。
金管會銀行局副局長黃光熙表示，目前國銀在香港營運情況，共有23家分支機構、71個據點，包括19家分行、1家子行、2家財務子公司、1家保險經紀子公司。
根據金管會統計，到今年3月底為止，在香港總資產達2兆4,443億元，存款餘額1兆7,963億元，放款餘額則來到1兆17億元。
統計也指出，到今年5月為止，國內銀行在香港曝險金額為6,748億元，其中授信總額為5,149億元，投資總額則是1,599億元，雙雙較去年同期的5,628億元、1,701億元來得減少。
觀察國銀香港曝險情況，今年以來，國銀香港曝險3月一度突破7,000億元關卡，來到7,075億元，但到了4月則降至6,877億元，而5月則持續下滑至6,748億元。
市場關注國銀在香港曝險金額有逐步下降的原因為何，黃光熙指出，國銀在港曝險金額雖有下降的趨勢，但是否因港版國安法所影響則有待觀察。至於是否影響香港金融中心的地位，將會持續關注，目前尚未有

財經中心／台北報導
大同（2371）今（30）日舉行股東常會改選董事，最後由公司派守住經營權，6席董事、3席獨董合計9席董事全拿，林郭文艷保住經營權，但全軍覆沒的市場派，也對股東會提出諸多質疑抗議。在金管會要求下，大同公司今晚7點30分，要召開重大訊息說明會，《ETtoday新聞雲》將全程直播，帶您一起瞭解大同公司如何回應。
今天的股東會由於攸關經營權之爭，公司派與市場派兩路人馬互不相讓，會場出現大批身穿黑衣的保全人員嚴格管制進出，場內則是吵成一團，市場派質疑有2萬6000張股票的投票權遭公司派刪除，但公司派並未理會，依舊進行投票表決，董監改選結果出爐，公司派完勝，市場派一席未得，包括角逐獨立董事的前立委黃國昌也落選。
▲大同公司6/30舉行109年股東常會，公司派市場派大戰再起。市場派代表參選獨董的前立委黃國昌一早就來會場，並對媒體發表談話。（圖／記者湯興漢攝）
市場派強烈表達不滿與抗議，隨後在下午4點召開記者會，呼籲經濟部跟金管會發揮主管機關監督功能，不要准許新任董事登記，並且揚言要提告訴諸司法。
市場派表示，大同公司發行股數全部23.4億股，林郭文艷董事長自行決定封了13億股，超過一半的股票都被沒收。三圓建設董事長王光祥表示，大同公司實在是目無法紀，只要不是支持林郭文艷的人，大同公司都把他們的股權刪除，導致股東沒有選舉票，因此呼籲主管機關能夠嚴正執法，否則會讓台灣資本市場痛心，這次臺灣防疫成功，如果資本市場能健全，台灣才能夠達到全世界有目共睹的目標。
▲大同經營權之爭延燒，市場派在股東會後召開臨時記者會，其中包括三圓建設董事長王光祥（圖／記者湯興漢攝）
市場派代表莊正律師指出，參與這場股東會心情沈重，自己處理很多起經營權案，但今天是最黑暗公司治理情況，現場黑衣人讓股東像是到了恐怖境地，對小股東而言，心裡非常有壓力。此外，大同這次投票票匭只有一個，逐案表決根本沒做到，關鍵問題在沒有主管機關命令下，直接認定誰可以投票，因此呼籲經濟部跟金關會發揮監督功能，讓資本市場交易秩序可以維持，也希望經濟部允許股東自行召開臨時股東會，以維護股東們的權益。
莊正律師指出，股東會有很多侵害股東權的情況，是史無前例的，這會造成日後非常嚴重的影響，希望主管機關撤銷大同公司的自辦股務。
▲大同股東會。（圖／大同股東提供）
